In [1]:
import numpy as np
import pandas as pd
import json as js
from functions import dump_links_json, dump_dish_json

In [4]:
def split_data(s):
    try:
        string = s.split(':')
        if len(string) >= 2:
            return string[0], string[1]
        else:
            return s, None
    except ValueError:
        return s, None


In [ ]:
import requests
from bs4 import BeautifulSoup

links = []

for page_number in range(2, 59):
    url = f"https://bepmina.vn/mon-viet/page/{page_number}"

    try:
        response = requests.get(url)
        response.raise_for_status()  

        webpage = BeautifulSoup(response.text, 'html.parser')
        content = webpage.find_all('a', class_="more-link")

        for link in content:
            links.append(link.get('href'))

    except requests.exceptions.RequestException as e:
        print(f"Error encountered for page {page_number}: {e}")
    print(page_number)

dump_links_json(links)


In [ ]:
with open('./group-project/links.json','r',encoding='utf-8') as f:
    data = js.load(f)

In [ ]:
for dish in data['link']:
    url = dish
    title = str()
    ingredients = []
    quantities = []
    error_links = []
    try:
        response = requests.get(url)
        response.raise_for_status()

        webpage = BeautifulSoup(response.text, 'html.parser')
        title = (webpage.find('h1').get_text())
        ul_tags = webpage.find_all('ul',class_="ingredients-list")
        for ul_tag in ul_tags:
            li_tags = ul_tag.find_all('li')
            for i in li_tags:
                string = i.get_text()
                ingredient, quantity = split_data(string)
                ingredients.append(ingredient)
                quantities.append(quantity)
    except requests.exceptions.RequestException as e:
        print(f"Error encountered for page {url}: {e}")
    dump_dish_json(title,ingredients,quantities)